In [36]:
import random
import numpy as np
from random import choice

# Определяем расширенное поле 
K = GF(89, 'alpha')
alpha = K.gen()    

def check_inequality(n, k, r):
    half_k = k * (k + 1) // 2
    return (r == n) if half_k >= n else (r == half_k)

def make_weight1(n, K):
    slots = list(range(n))
    P = matrix(K, n, n, 0)
    for i in range(n):
        index = choice(slots)
        slots.remove(index)
        P[i, index] = 1
    return P

def Schur_product(M, k1):
    Schur = []
    for i in range(k1):
        row_i = np.array(list(M[i]))
        for j in range(i, k1):
            row_j = np.array(list(M[j]))
            Schur.append(list(np.multiply(row_i, row_j)))
    return Schur

def generate_random_non_singular_matrix(K, k1):
    S = random_matrix(K, k1, k1)
    while S.is_singular():
        S = random_matrix(K, k1, k1)
    return S

def SchurCalc(M, K):
    k1 = M.rank()
    n = M.ncols()
    S = generate_random_non_singular_matrix(K, k1)
    P1 = make_weight1(n, K)
    G_cap_hidden1 = S * M * P1
    G_cap_Schur1 = Matrix(K, Schur_product(G_cap_hidden1, k1))
    G1k = G_cap_Schur1.rank()
    return check_inequality(n, k1, G1k)

def codegen(K):
    """
    Строим AG-код на кривой квартика Клейна:
        x^3 y + y^3 z + z^3 x = 0.
    """
    res = [None] * 5

    # Определяем квартику Клейна: уравнение проективной формы
    R.<x, y> = PolynomialRing(K, 2)
    f = x^3 * y + y^3 + x
    C_affine = Curve(f)

    # Получаем функциональное поле и места порядка 1
    F_field = C_affine.function_field()
    places = F_field.places(1)

    print(f"Квартика Клейна: {f}, число мест = {len(places)}")

    if len(places) <= 1:
        raise ValueError("Недостаточно мест для построения AG-кода")

    # Формируем дивизор G = d * Q, где Q – первая найденная точка
    Q = places[0]
    del places[0]
    d = len(places) // 2
    G = d * Q

    # AG-код
    code = codes.EvaluationAGCode(places, G)
    res[0] = SchurCalc(code.generator_matrix(), K)

    # Проколотый код 
    try:
        code1 = codes.PuncturedCode(code, 1)
        res[1] = SchurCalc(code1.generator_matrix(), K)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить проколотый")

    # Укороченный код
    try:
        code2 = code1.shortened([1])
        res[2] = SchurCalc(code2.generator_matrix(), K)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить укороченный")

    # Subfield-subcode
    code3 = codes.SubfieldSubcode(code, K)
    res[3] = SchurCalc(code3.generator_matrix(), K)

    # Trace-code
    code41 = code.dual_code()
    code42 = codes.SubfieldSubcode(code41, K)
    code43 = code42.dual_code()
    res[4] = SchurCalc(code43.generator_matrix(), K)

    return res

# -------------------- ОСНОВНОЙ БЛОК --------------------
number_tests = 0   # Количество успешных тестов
probs = [0] * 5    # Средние значения (доли успеха) для пяти типов кодов
total_tests = 15  # Количество попыток

while number_tests < total_tests:
    try:
        res = codegen(K)
        if None not in res:
            for i in range(5):
                probs[i] = (probs[i] * number_tests + (1 if res[i] else 0)) / (number_tests + 1)
            number_tests += 1
    except Exception as e:
        print("error is", e)
        continue

code_names = ['AG-code', 'Punctured-code', 'Shortened-code', 'Subfield-code', 'Trace-code']
for i in range(5):
    print(f"{code_names[i]}: {probs[i]}")



error is error in Singular function call 'groebner':
no ring active (9)
// ** redefining i (ideal i=i_par; ) standard.lib::groebner:872
// ** redefining Qideal (  ideal Qideal = ideal(P);      //defining the quotient ideal if P is a qring) standard.lib::groebner:932
// ** redefining BRlist (  list BRlist = ring_list(P);     //ringlist of basering) standard.lib::groebner:940


KeyboardInterrupt: 